In [2]:
from itertools import combinations

def load_dataset():
    return [
        ['milk', 'bread', 'butter'],
        ['bread', 'butter'],
        ['milk', 'bread'],
        ['milk', 'butter'],
        ['bread', 'butter'],
    ]

def get_frequent_itemsets(dataset, min_support):
    item_counts = {}
    for transaction in dataset:
        for item in transaction:
            item_counts[item] = item_counts.get(item, 0) + 1

    num_transactions = len(dataset)
    frequent_items = {
        (item,): count / num_transactions
        for item, count in item_counts.items()
        if count / num_transactions >= min_support
    }
    return frequent_items

def has_infrequent_subset(candidate, prev_freq_itemsets):
    k = len(candidate)
    subsets = list(combinations(candidate, k - 1))
    return any(tuple(sorted(sub)) not in prev_freq_itemsets for sub in subsets)

def generate_candidates(prev_freq_itemsets, k):
    candidates = []
    items = list(prev_freq_itemsets.keys())
    for i in range(len(items)):
        for j in range(i + 1, len(items)):
            l1 = list(items[i])
            l2 = list(items[j])
            l1.sort()
            l2.sort()
            if l1[:k - 2] == l2[:k - 2]:
                candidate = tuple(sorted(set(l1) | set(l2)))
                if not has_infrequent_subset(candidate, prev_freq_itemsets):
                    candidates.append(candidate)
    return candidates

def apriori(dataset, min_support):
    dataset = list(map(set, dataset))
    freq_itemsets = {}

    one_itemsets = get_frequent_itemsets(dataset, min_support)
    freq_itemsets.update(one_itemsets)

    k = 2
    current_freq_itemsets = one_itemsets

    while current_freq_itemsets:
        candidates = generate_candidates(current_freq_itemsets, k)
        candidate_counts = {c: 0 for c in candidates}

        for transaction in dataset:
            for candidate in candidates:
                if set(candidate).issubset(transaction):
                    candidate_counts[candidate] += 1

        num_transactions = len(dataset)
        current_freq_itemsets = {
            itemset: count / num_transactions
            for itemset, count in candidate_counts.items()
            if count / num_transactions >= min_support
        }

        freq_itemsets.update(current_freq_itemsets)
        k += 1

    return freq_itemsets

# Example usage
if __name__ == "__main__":
    dataset = load_dataset()
    min_support = 0.6
    freq_itemsets = apriori(dataset, min_support)

    print("Frequent itemsets:")
    for itemset, support in freq_itemsets.items():
        print(f"{itemset}: {support:.2f}")


Frequent itemsets:
('butter',): 0.80
('milk',): 0.60
('bread',): 0.80
('bread', 'butter'): 0.60
